In [1]:
import os
import random
import shutil
import glob
from typing import List, Dict, Set

# 💡 설정값들을 환경에 맞게 반드시 수정하세요!
# -------------------------------------------------------------
BASE_DIR = r'F:\ML\dataset'                 # 👈 모든 데이터 폴더의 상위 경로
SOURCE_DIR_NAME = 'Processed_224x224'  # 👈 전처리된 이미지가 저장된 폴더명
VALIDATION_DIR_NAME = 'Validation_Set' # 👈 검증 데이터셋을 저장할 새 폴더명
FILES_TO_MOVE_PER_GENRE = 3000         # 👈 장르별 목표 파일 수
# -------------------------------------------------------------

TARGET_TAGS: List[str] = [
    "Adventure", "Action", "RPG", "Strategy", "Simulation", "Sports", "Racing", 
    "Puzzle", "Sandbox", "Shooter", "Survival"
]

def get_file_count_and_sort_tags(source_base_dir: str) -> List[str]:
    """
    각 장르 폴더의 파일 수를 세어 데이터가 적은 장르부터 오름차순으로 정렬합니다.
    """
    tag_counts: Dict[str, int] = {}
    print("🔍 각 장르별 파일 수를 확인 중...")
    
    for tag in TARGET_TAGS:
        source_genre_dir = os.path.join(source_base_dir, tag)
        if os.path.isdir(source_genre_dir):
            # 이미지 파일 (jpg, png)만 카운트
            file_paths = glob.glob(os.path.join(source_genre_dir, '*.[pj][np]g'))
            tag_counts[tag] = len(file_paths)
        else:
            tag_counts[tag] = 0
            
    # 파일 수 오름차순으로 태그 정렬
    sorted_tags = sorted(tag_counts.keys(), key=lambda t: tag_counts[t])
    
    print("✅ 데이터가 적은 장르부터 순서대로 처리합니다:")
    for tag in sorted_tags:
        print(f"   - {tag}: {tag_counts[tag]}개")
        
    return sorted_tags

# 💡 새로운 헬퍼 함수: 학습 데이터셋에서 검증 파일과 중복되는 모든 파일 삭제
def remove_duplicates_from_all_train_dirs(
    base_dir: str, 
    source_dir_name: str, 
    filename_to_remove: str, 
    target_tags: List[str]
):
    """
    모든 장르 폴더를 순회하며 지정된 파일명과 동일한 파일을 삭제합니다.
    """
    source_base_dir = os.path.join(base_dir, source_dir_name)

    for tag in target_tags:
        train_genre_dir = os.path.join(source_base_dir, tag)
        file_path = os.path.join(train_genre_dir, filename_to_remove)
        
        if os.path.exists(file_path):
            try:
                os.remove(file_path)
                # print(f"    [중복 제거] {tag} 폴더에서 {filename_to_remove} 삭제 완료.")
            except Exception as e:
                print(f"    [중복 제거] ❌ 삭제 오류 ({file_path}): {e}")

def create_unique_validation_set():
    """
    각 장르에서 2000개의 고유한 파일을 랜덤 선택하여 검증 데이터셋으로 이동시킵니다.
    모든 검증 폴더를 통틀어 파일명이 중복되지 않도록 보장합니다.
    """
    
    source_base_dir = os.path.join(BASE_DIR, SOURCE_DIR_NAME)
    validation_base_dir = os.path.join(BASE_DIR, VALIDATION_DIR_NAME)
    os.makedirs(validation_base_dir, exist_ok=True)
    
    # 데이터가 적은 장르부터 처리하도록 태그 순서 정렬
    sorted_tags = get_file_count_and_sort_tags(source_base_dir)

    print("-" * 50)
    print(f"검증 데이터셋 분리 시작. 장르별 목표: {FILES_TO_MOVE_PER_GENRE}개")
    print("-" * 50)
    
    # 1. 전역 추적: 이미 검증 데이터셋에 포함된 모든 파일명 (중복 방지용)
    global_validation_filenames: Set[str] = set()
    
    # 기존에 이동된 파일이 있다면 목록에 추가하여 재실행 시 중복되지 않도록 함
    for tag in TARGET_TAGS:
        validation_genre_dir = os.path.join(validation_base_dir, tag)
        if os.path.isdir(validation_genre_dir):
            moved_files = glob.glob(os.path.join(validation_genre_dir, '*.[pj][np]g'))
            global_validation_filenames.update({os.path.basename(f) for f in moved_files})

    total_moved = len(global_validation_filenames)
    print(f"🚨 현재 검증 데이터셋에 이미 {total_moved}개의 고유한 파일이 있습니다. 작업을 이어갑니다.")

    # 2. 정렬된 순서대로 장르별 파일 이동 시작
    for tag in sorted_tags:
        source_genre_dir = os.path.join(source_base_dir, tag)
        validation_genre_dir = os.path.join(validation_base_dir, tag)
        os.makedirs(validation_genre_dir, exist_ok=True)
        
        # 3. 현재 장르 폴더에 이미 이동된 파일 수 확인
        current_moved_in_genre = len(glob.glob(os.path.join(validation_genre_dir, '*.[pj][np]g')))
        
        if current_moved_in_genre >= FILES_TO_MOVE_PER_GENRE:
            print(f"[{tag}] ✅ 목표 수 ({FILES_TO_MOVE_PER_GENRE}개) 달성 완료. 스킵합니다.")
            continue

        needed_to_move = FILES_TO_MOVE_PER_GENRE - current_moved_in_genre
        
        # 4. 원본 폴더에서 이동 가능한 모든 파일 목록 가져오기
        all_source_files = glob.glob(os.path.join(source_genre_dir, '*.[pj][np]g'))
        
        # 5. 전역 중복을 확인하여 이동 후보 파일 목록 생성
        # (아직 검증 데이터셋의 어떤 폴더에도 들어가지 않은 파일만 후보가 됨)
        unique_candidates = [
            f for f in all_source_files 
            if os.path.basename(f) not in global_validation_filenames
        ]

        # 6. 랜덤 선택 및 이동
        if not unique_candidates:
            print(f"[{tag}] ⚠️ 이동 가능한 고유 파일이 부족하거나 남아있지 않습니다. (현재: {current_moved_in_genre}개)")
            continue
            
        if len(unique_candidates) < needed_to_move:
            # 원본에 남아있는 고유 파일이 부족한 경우
            selected_files = unique_candidates
            print(f"[{tag}] ⚠️ 파일 부족: {len(selected_files)}개만 이동합니다.")
        else:
            # 필요한 파일 수만큼 랜덤 선택
            selected_files = random.sample(unique_candidates, needed_to_move)

        moved_count = 0
        
        for source_path in selected_files:
            filename = os.path.basename(source_path)
            destination_path = os.path.join(validation_genre_dir, filename)
            
            try:
                # 파일을 원본 폴더에서 검증 폴더로 이동 (원본 폴더에서는 삭제됨)
                shutil.move(source_path, destination_path)
                
               # 2. 💡 핵심 추가 로직: 나머지 모든 장르 폴더에서 동일 파일명을 삭제
                remove_duplicates_from_all_train_dirs(
                    BASE_DIR, 
                    SOURCE_DIR_NAME, 
                    filename, 
                    TARGET_TAGS
                )
                
                # 3. 전역 추적 셋에 파일명 추가 및 카운트
                global_validation_filenames.add(filename)
                moved_count += 1
            except Exception as e:
                print(f"[{tag}] ❌ 파일 이동 오류 ({filename}): {e}")

        print(f"[{tag}] 이동 완료. 총 {current_moved_in_genre + moved_count}개 확보 (이번에 {moved_count}개 이동).")

    print("-" * 50)
    print(f"🎉 모든 장르 분리 완료.")
    print(f"최종적으로 검증 데이터셋에 확보된 고유 이미지 수: {len(global_validation_filenames)}장")
    print(f"검증 데이터셋 경로: {validation_base_dir}")
    print("-" * 50)

# 함수 실행
create_unique_validation_set()

🔍 각 장르별 파일 수를 확인 중...
✅ 데이터가 적은 장르부터 순서대로 처리합니다:
   - Sports: 18948개
   - Racing: 18949개
   - Sandbox: 19044개
   - Survival: 28665개
   - Shooter: 28975개
   - Simulation: 38160개
   - Strategy: 38687개
   - RPG: 38801개
   - Puzzle: 39081개
   - Action: 57488개
   - Adventure: 58005개
--------------------------------------------------
검증 데이터셋 분리 시작. 장르별 목표: 3000개
--------------------------------------------------
🚨 현재 검증 데이터셋에 이미 22000개의 고유한 파일이 있습니다. 작업을 이어갑니다.
[Sports] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
[Racing] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
[Sandbox] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
[Survival] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
[Shooter] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
[Simulation] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
[Strategy] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
[RPG] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
[Puzzle] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
[Action] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
[Adventure] 이동 완료. 총 3000개 확보 (이번에 1000개 이동).
--------------------------------------------------
🎉 모든 장